In [ ]:
!pip install tensorflow

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (757 kB)
  Using cached regex-2023.8.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (782 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)


In [1]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

/home/hongquan/Projects/3_nlp-huggingface/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

Trong ví dụ trên Pipeline, đã nhóm 3 bước lại với nhau: tiền xử lý data, đưa data qua mô hình và hậu xử lý

### Preprocessing with a tokenizer

**Quy trình**
- Tách input thành words, subwords, symbols(như dấu câu) được gọi là các token
- Mapping mỗi token thành số (int)
- Thêm các đầu vào bổ sung

- Tất cả quá trình tiền xử lý cần được thực hiện giống như khi model được train, nên đầu tiên cấn tải thông tin đó từ Model Hub
- Sử dụng class AutoTokenizer với phương thức from_pretrained(), tokenizer sẽ được lưu vào cache

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Sau khi có tokenizer, chúng tôi có thể trực tiếp chuyển các câu của mình tới nó và chúng tôi sẽ trả lại một từ điển sẵn sàng cung cấp cho mô hình! Điều duy nhất còn lại phải làm là chuyển đổi danh sách input ID thành tensor.

Để chỉ định loại tensor mà chúng ta muốn trả về (PyTorch, TensorFlow hoặc NumPy), chúng ta sử dụng đối số return_tensors:

In [5]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


Output là 1 dictionary bao gồm:
- input_ids: Là 1 tensor gôm 2 hàng số nguyên, mỗi hàng tương ứng với 1 câu
- attention_mask: Giải thích sau

### Going through the model

In [9]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

- Output được gọi là hidden state (trạng thái ẩn), còn được gọi là features (tính năng).
- Với mỗi input, output sẽ là  1 high-dimensional vector 

**A high-dimensional vector?**

Vector output của Transformer thường lớn. nó có 3 chiều
- Batch size: Số lượng sequences được xử lý tại cùng 1 thời điểm (Trong ví dụ là 2)
- Sequence length: Độ dài của biểu diễn số trong 1 chuỗi (Trong ví dụ là 16)
- Hidden size: Chiều của mỗi vector input

In [16]:
outputs = model(inputs["input_ids"])
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


### Model heads: Making sense out of numbers

### Postprocessing the output

Bản thân các giá trị nhận được dưới dạng đầu ra từ mô hình của mình không nhất thiết phải có ý nghĩa.

In [17]:
print(outputs.logits)

AttributeError: 'BaseModelOutput' object has no attribute 'logits'